<a href="https://colab.research.google.com/github/bill-wilkin/Prediction-of-Product-Sales/blob/main/prediction_of_product_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prediction of Product Sales
- Bill Wilkin


##Project Overview

##Load and Inspect Data

##Clean Data

##Exploratory Data Analysis

##Exploratory Data Analysis